In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df_main = pd.read_csv("nlz_art11.csv", delimiter = "|", header = None)

In [4]:
#taking the kürzel in the beginning to the author section
tester2 = df_main.copy()
tester2[13] = df_main[5].astype(str)

tester2[13] = tester2[13].str[:10]

tester2[14] = tester2[13].apply(lambda x: x[1:4] if x[0] == "(" else "")


df_main[4] = np.where(tester2[14] != "", tester2[14], df_main[4])

In [20]:
#continuation of the above
tester2[5] = tester2[5].astype("str")
tester2[6] = tester2[5].apply(lambda x: x[5:] if x[0] == "(" else "")
df_main[5] = np.where(tester2[6] != "", tester2[6], tester2[5])

In [21]:
#remove columns without title, text or url
df_main = df_main.dropna(subset=[0])
df_main = df_main.dropna(subset=[5])
df_main = df_main.dropna(subset=[6])

In [22]:
#set date and remove all entries from 2021
df_main[2] = pd.to_datetime(df_main[2], format="%d/%m/%Y")
df_main[7] = df_main[2].dt.year
df_main = df_main[df_main[7] != 2021]
df_main = df_main.drop([7], axis =1)

In [23]:
#drop duplicates
df_main = df_main.drop_duplicates(keep = "last")

In [24]:
#extracting categories
categories = df_main[6].str.split("/",expand=True)
categories[6] = categories[4].str.len()
categories[7] = categories[6].astype(int)
categories[8] = np.where(categories[7] < 20, categories[4],"")

df_main[8] = categories[8]
df_main[7] = categories[3]

In [26]:
#replace topics with ressorts
df_main[9] = df_main[7]
df_main[10] = df_main[8]

with open("ressorts_cat.bin", "rb") as data:
    ressorts = pickle.load(data)

df_main[9].replace(ressorts, inplace=True)
df_main[10].replace(ressorts, inplace=True)

In [27]:
#delete unworkables
with open("to_delete.bin", "rb")as data:
    to_delete = pickle.load(data)
    
df_main = df_main[~df_main[8].isin(to_delete)]
df_main = df_main[~df_main[7].isin(to_delete)]

In [28]:
#remove double indication of the same category
df_main[11] = np.where(df_main[9] == df_main[10], "", df_main[10])
df_main = df_main.drop(columns = 10)

In [29]:
df_main[4] = df_main[4].str.replace("(", "")
df_main[4] = df_main[4].str.replace(")", "")
df_main[4] = df_main[4].str.replace("Covid-19 über alles.", "")

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
#reset index
df_main = df_main.reset_index(drop=True)

,0,1,2,3,4,5,6,8,7,9,11
0,Leserbrief Covid-19: Wir haben es übertrieben,"«So geht es Europas Corona-Extremen», Ausgabe ...",2020-07-09,17.00,NaN,"Zum Glück gibt es Schweden. Ein Land, das nich...",https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,
1,Leserbrief Covid-19: Wenn wir im Bus hüsteln,Zur Coronakrise,2020-10-22,16.42,NaN,"Covid-19 und die Massnahmen zeigen, was mit de...",https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,
2,Langlauf: Covid-19 und Leistungssport,"«Norweger als Totengräber des Langlaufs?», Aus...",2020-12-15,15.17,NaN,Mit Erstaunen habe ich diesen Bericht gelesen....,https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,
3,Leserdebatte Covid-19: Enttäuscht vom Bunde...,"«Ringen um neue Corona­verschärfungen», Ausgab...",2020-10-16,16.26,NaN,Die Medienkonferenz des Bundesrates wurde drin...,https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,
4,Covid-19: Dankbarkeit und Zweifel,Zur Coronakrise,2020-12-22,15.39,NaN,"Ich, geboren 1940, habe beim Lesen und Hören i...",https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,
...,...,...,...,...,...,...,...,...,...,...,...
1451,In Uri gibt es aktuell fünf Coronafälle,Im Bulletin vom 31. Juli ist die Rede von 35 K...,2020-07-31,12.47,Florian Arnold,"Mit dem Bulletin vom 31. Juli 2020, 11.00 Uhr,...",https://www.luzernerzeitung.ch/zentralschweiz/...,uri,zentralschweiz,lokales,
1452,Der Buochser Jodlerausflug endet in der Quaran...,Eine Jodlerin steckt sich wohl unwissentlich m...,2020-02-10,18.13,Matthias Piazza,Der Jodlerklub Heimelig Buochs liess sich wege...,https://www.luzernerzeitung.ch/zentralschweiz/...,nidwalden,zentralschweiz,lokales,
1453,Nach erfolgreichem Geschäftsjahr: Verantwortli...,Die Führungsriege der Bank ist zufrieden mit d...,2020-05-05,18.44,Pascal Studer,Vergangene Woche haben die Delegierten der Rai...,https://www.luzernerzeitung.ch/zentralschweiz/...,nidwalden,zentralschweiz,lokales,
1454,100 Nidwaldner Jugendliche im Kantonslager in ...,Das beliebte Tenerolager für Nidwaldner Schüle...,2020-08-13,16.38,Matthias Piazza,Noch bis am Samstag versuchen sich gut 100 Nid...,https://www.luzernerzeitung.ch/zentralschweiz/...,nidwalden,zentralschweiz,lokales,


In [31]:
df_main[20] = "nlz"
df_main[4] = df_main[4].str.replace("http\S+", "",regex = True)
df_main[4] = df_main[4].str.replace("Bild", "")

In [32]:
df_main

,0,1,2,3,4,5,6,8,7,9,11,20
3,Leserbrief Covid-19: Wir haben es übertrieben,"«So geht es Europas Corona-Extremen», Ausgabe ...",2020-07-09,17.00,NaN,"Zum Glück gibt es Schweden. Ein Land, das nich...",https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
4,Leserbrief Covid-19: Wenn wir im Bus hüsteln,Zur Coronakrise,2020-10-22,16.42,NaN,"Covid-19 und die Massnahmen zeigen, was mit de...",https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
5,Langlauf: Covid-19 und Leistungssport,"«Norweger als Totengräber des Langlaufs?», Aus...",2020-12-15,15.17,NaN,Mit Erstaunen habe ich diesen Bericht gelesen....,https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
6,Leserdebatte Covid-19: Enttäuscht vom Bunde...,"«Ringen um neue Corona­verschärfungen», Ausgab...",2020-10-16,16.26,NaN,Die Medienkonferenz des Bundesrates wurde drin...,https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
7,Covid-19: Dankbarkeit und Zweifel,Zur Coronakrise,2020-12-22,15.39,NaN,"Ich, geboren 1940, habe beim Lesen und Hören i...",https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
...,...,...,...,...,...,...,...,...,...,...,...,...
1577,In Uri gibt es aktuell fünf Coronafälle,Im Bulletin vom 31. Juli ist die Rede von 35 K...,2020-07-31,12.47,Florian Arnold,"Mit dem Bulletin vom 31. Juli 2020, 11.00 Uhr,...",https://www.luzernerzeitung.ch/zentralschweiz/...,uri,zentralschweiz,lokales,,nlz
1579,Der Buochser Jodlerausflug endet in der Quaran...,Eine Jodlerin steckt sich wohl unwissentlich m...,2020-02-10,18.13,Matthias Piazza,Der Jodlerklub Heimelig Buochs liess sich wege...,https://www.luzernerzeitung.ch/zentralschweiz/...,nidwalden,zentralschweiz,lokales,,nlz
1580,Nach erfolgreichem Geschäftsjahr: Verantwortli...,Die Führungsriege der Bank ist zufrieden mit d...,2020-05-05,18.44,Pascal Studer,Vergangene Woche haben die Delegierten der Rai...,https://www.luzernerzeitung.ch/zentralschweiz/...,nidwalden,zentralschweiz,lokales,,nlz
1581,100 Nidwaldner Jugendliche im Kantonslager in ...,Das beliebte Tenerolager für Nidwaldner Schüle...,2020-08-13,16.38,Matthias Piazza,Noch bis am Samstag versuchen sich gut 100 Nid...,https://www.luzernerzeitung.ch/zentralschweiz/...,nidwalden,zentralschweiz,lokales,,nlz


In [33]:
df_main[4] = df_main[4].str.strip()
df_main[5] = df_main[5].str.strip()
df_main

,0,1,2,3,4,5,6,8,7,9,11,20
3,Leserbrief Covid-19: Wir haben es übertrieben,"«So geht es Europas Corona-Extremen», Ausgabe ...",2020-07-09,17.00,NaN,"Zum Glück gibt es Schweden. Ein Land, das nich...",https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
4,Leserbrief Covid-19: Wenn wir im Bus hüsteln,Zur Coronakrise,2020-10-22,16.42,NaN,"Covid-19 und die Massnahmen zeigen, was mit de...",https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
5,Langlauf: Covid-19 und Leistungssport,"«Norweger als Totengräber des Langlaufs?», Aus...",2020-12-15,15.17,NaN,Mit Erstaunen habe ich diesen Bericht gelesen....,https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
6,Leserdebatte Covid-19: Enttäuscht vom Bunde...,"«Ringen um neue Corona­verschärfungen», Ausgab...",2020-10-16,16.26,NaN,Die Medienkonferenz des Bundesrates wurde drin...,https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
7,Covid-19: Dankbarkeit und Zweifel,Zur Coronakrise,2020-12-22,15.39,NaN,"Ich, geboren 1940, habe beim Lesen und Hören i...",https://www.luzernerzeitung.ch/meinung/leserbr...,leserbriefe,meinung,meinung,,nlz
...,...,...,...,...,...,...,...,...,...,...,...,...
1577,In Uri gibt es aktuell fünf Coronafälle,Im Bulletin vom 31. Juli ist die Rede von 35 K...,2020-07-31,12.47,Florian Arnold,"Mit dem Bulletin vom 31. Juli 2020, 11.00 Uhr,...",https://www.luzernerzeitung.ch/zentralschweiz/...,uri,zentralschweiz,lokales,,nlz
1579,Der Buochser Jodlerausflug endet in der Quaran...,Eine Jodlerin steckt sich wohl unwissentlich m...,2020-02-10,18.13,Matthias Piazza,Der Jodlerklub Heimelig Buochs liess sich wege...,https://www.luzernerzeitung.ch/zentralschweiz/...,nidwalden,zentralschweiz,lokales,,nlz
1580,Nach erfolgreichem Geschäftsjahr: Verantwortli...,Die Führungsriege der Bank ist zufrieden mit d...,2020-05-05,18.44,Pascal Studer,Vergangene Woche haben die Delegierten der Rai...,https://www.luzernerzeitung.ch/zentralschweiz/...,nidwalden,zentralschweiz,lokales,,nlz
1581,100 Nidwaldner Jugendliche im Kantonslager in ...,Das beliebte Tenerolager für Nidwaldner Schüle...,2020-08-13,16.38,Matthias Piazza,Noch bis am Samstag versuchen sich gut 100 Nid...,https://www.luzernerzeitung.ch/zentralschweiz/...,nidwalden,zentralschweiz,lokales,,nlz


In [34]:
df_main.to_csv("nlz_artikel.csv", header = None, sep = "|", index = False)